## Set up
First you need to install GraphDB locally on you machine


In [5]:
from rdflib import ConjunctiveGraph
from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import numpy as np
from IPython.display import display, HTML


import pandas as pd

def remoteQuery(query, endpoint):
    endpoint.setQuery(query)
    try:
        result = endpoint.queryAndConvert()
        pd.set_option("display.max_rows",None,"display.max_colwidth",6000,"display.width",6000,)
        df = pd.DataFrame(result['results']['bindings'])
        df = df.applymap(lambda x: x['value'])
        return df
        #return (result['results']['bindings'])
    except Exception as e:
        print(e)


In [6]:
ep_biotools = SPARQLWrapper("http://localhost:7200/repositories/Project25")
ep_biotools.setReturnFormat(JSON)

## List all EDAM entities with relations (transitive)

You can change the relation in the query in   `VALUES ?relation { xxxxxxx }` to : has_topic, has_output, has_input, is_format_of

In [11]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT
    ?entity ?label
    (COUNT(?target) as ?count)
WHERE
{
    VALUES ?relation { edam:has_topic }
    ?entity
        rdfs:subClassOf [owl:onProperty ?relation ; owl:someValuesFrom ?target] ;
        rdfs:label ?label .
    ?target rdfs:label ?target_label .
} GROUP BY ?entity ?label
ORDER BY DESC(?count) ASC(?entity) ASC(?target)
"""


In [12]:
results1=remoteQuery(query=q, endpoint=ep_biotools)
print(f"nb of edam format with is_format_of relation: {len(results1)}")

display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results1.to_html() +
             "</div>"))

nb of edam format with is_format_of relation: 618


,entity,label,count
0,http://edamontology.org/operation_0269,Transmembrane protein prediction,8
1,http://edamontology.org/operation_0245,Structural motif discovery,7
2,http://edamontology.org/operation_0246,Protein domain recognition,7
3,http://edamontology.org/operation_0267,Protein secondary structure prediction,7
4,http://edamontology.org/operation_0268,Protein super-secondary structure prediction,7
5,http://edamontology.org/operation_0390,Protein peeling,7
6,http://edamontology.org/operation_0468,Protein secondary structure prediction (helices),7
7,http://edamontology.org/operation_0469,Protein secondary structure prediction (turns),7
8,http://edamontology.org/operation_0470,Protein secondary structure prediction (coils),7
9,http://edamontology.org/operation_2464,Protein-protein binding site prediction,7


## EDAM Relations consistent with boi.tools entries